In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils #use to convert class vectors to binary class matrices

Using Theano backend.


In [2]:
batch_size = 10
number_epoch = 200
number_class = 10
data_augmentation = True

In [3]:
img_row, img_col = 32, 32
img_RGB = 3

In [4]:
(X_train, y_train),(X_test, y_test) = cifar10.load_data()

In [5]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(y_test.shape[0], 'test samples')

X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, number_class)
Y_test = np_utils.to_categorical(y_test, number_class)

In [7]:
model = Sequential()

In [8]:
#first layer
model.add(Convolution2D(20, 2, 2,
                        border_mode = 'same',
                        input_shape = X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size = (3,3)))

#second layer
model.add(Convolution2D(40, 2, 2,
                        border_mode = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size = (3,3)))

#third layer
model.add(Convolution2D(60, 2, 2,
                       border_mode = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size = (2, 2)))


#Flatten1
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(number_class))
model.add(Activation('softmax'))

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '827' (I am process '785')


In [10]:
# model using SGD or adam + momentum and decay with learning rate.
sgd = SGD(lr=0.01, decay=0.0005, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=number_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=number_epoch,
                        validation_data=(X_test, Y_test))

Using real-time data augmentation.
Epoch 1/200
 2610/50000 [>.............................] - ETA: 122s - loss: 2.2848 - acc: 0.1153

KeyboardInterrupt: 

In [11]:
score = model.evaluate(X_test, Y_test)

10000/10000 [==============================] - 9s     
